In [20]:
import pandas as pd
import numpy as np
np.set_printoptions(suppress=True)
import scipy.fftpack

SESSIONPATH = 'Training Data/Session01/'
ARMDATA = 'armIMU.txt'
WRISTDATA = 'wristIMU.txt'
TIMEDATA = 'time.txt'


def load_data(filepath):
    
    arm = np.loadtxt(filepath + ARMDATA)
    wrist = np.loadtxt(filepath + WRISTDATA)
    time = np.loadtxt(filepath + TIMEDATA)
    #data = pd.DataFrame({'time':time, 'arm1':arm[:,1], 'wrist':wrist[:,1]})
    dataArm = pd.DataFrame(arm)
    dataWrist = pd.DataFrame(wrist)
    
    print(dataArm.shape)
    print(dataWrist.shape)
    
    return dataArm, dataWrist    

In [21]:
armData, wristData = load_data(SESSIONPATH)

(254455, 6)
(254455, 6)


In [22]:
def split_data(data):
    dataAcc = pd.DataFrame([[0,0,0]])
    dataAcc = dataAcc.append([dataAcc]*198, ignore_index=True)
    dataGyr = dataAcc
    
    dataAcc = dataAcc.append(data.loc[:,'0':'2'], ignore_index=True)

    gyrData = data.loc[:,'3':'5']
    gyrData.columns = [0,1,2]
    dataGyr = dataGyr.append(gyrData.loc[:,'0':'2'], ignore_index=True)
    
    return dataAcc, dataGyr

In [23]:
armDataAcc, armDataGyr = split_data(armData)
wristDataAcc, wristDataGyr = split_data(wristData)

In [24]:
print(armDataAcc)
print(armDataGyr)
print(wristDataAcc)
print(wristDataGyr)

            0      1      2
0       0.000  0.000  0.000
1       0.000  0.000  0.000
2       0.000  0.000  0.000
3       0.000  0.000  0.000
4       0.000  0.000  0.000
5       0.000  0.000  0.000
6       0.000  0.000  0.000
7       0.000  0.000  0.000
8       0.000  0.000  0.000
9       0.000  0.000  0.000
10      0.000  0.000  0.000
11      0.000  0.000  0.000
12      0.000  0.000  0.000
13      0.000  0.000  0.000
14      0.000  0.000  0.000
15      0.000  0.000  0.000
16      0.000  0.000  0.000
17      0.000  0.000  0.000
18      0.000  0.000  0.000
19      0.000  0.000  0.000
20      0.000  0.000  0.000
21      0.000  0.000  0.000
22      0.000  0.000  0.000
23      0.000  0.000  0.000
24      0.000  0.000  0.000
25      0.000  0.000  0.000
26      0.000  0.000  0.000
27      0.000  0.000  0.000
28      0.000  0.000  0.000
29      0.000  0.000  0.000
...       ...    ...    ...
254624 -0.845  0.401  0.481
254625 -0.847  0.415  0.485
254626 -0.851  0.398  0.469
254627 -0.842  0.368

In [25]:
def feature_extract(data):
    feature1 = data.rolling(200).mean().dropna();
    feature2 = data.rolling(200).cov().dropna();
    feature3 = data.rolling(200).skew().dropna();
    feature4 = data.rolling(200).kurt().dropna();

    feature1 = feature1.values
    feature2 = feature2.unstack(level=-1).values
    feature2 = np.delete(feature2,[3,6,7],1)
    feature3 = feature3.values
    feature4 = feature4.values
 
    fs = 50
    feature5 = []
    feature6 = []
    for i in range(0,data.shape[0]-199):
        featureValues =data.iloc[i:199+i].values;
        f5 = []
        f6 = []
        for i in range(0,3):
            g=abs(np.fft.fft(featureValues[:,i]))
            g = g[0:round(len(g)/2)];
            g[0] = 0
            l1 = np.arange(len(g));
            l1 = fs*l1;
            w = np.divide(l1,(2*len(g)))
            v = max(g)
            idx = np.argmax(g)
            f5.append(v)
            f6.append(w[idx])
        feature5.append(f5)
        feature6.append(f6)

    features = np.concatenate((feature1,feature2,feature3,feature4,feature5,feature6),axis=1)

    return features

In [26]:
features1 = feature_extract(armDataAcc)
features2 = feature_extract(armDataGyr)
features3 = feature_extract(wristDataAcc)
features4 = feature_extract(wristDataGyr)

features = np.concatenate((features1,features2,features3,features4),axis=1)

print(len(features))

254455


In [27]:
print(len(features[5]))
print(type(features))
features1[5]

84
<class 'numpy.ndarray'>


array([-0.02407   ,  0.01606   ,  0.01116   ,  0.01882896, -0.01255763,
       -0.00872995,  0.00839432,  0.00581931,  0.00405076, -5.55305831,
        5.56561523,  5.55986637, 29.13143065, 29.32165899, 29.23491705,
        3.99400306,  2.70735776,  1.85013489,  0.25      ,  0.25      ,
        0.25      ])

In [45]:
np.savetxt('features.csv', features, fmt ='%f' ,delimiter = ',')